# Skelettschätzer

In [1]:
!pip install ipywidgets tomli ray
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

import os
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output, HTML
import ray
import subprocess
import yaml
import uuid

Defaulting to user installation because normal site-packages is not writeable
(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/home/fese4852/.local/lib/python3.8/site-packages/jupyterlab/debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/home/fese4852/.local/lib/python3.8/site-packages/jupyterlab/debuglog.py:55: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/home/fese4852/.local/lib/python3.8/site-packages/jupyterlab/debuglog.py:56: UserWarning: See the log file for 

ℹ Bitte aktivieren Sie die Logs, denn einige Ausgaben können nur darüber angezeigt werden.
`View > Show Log Console`

## 1. Auswählen von Videos
Wählen Sie zunächst Videos aus, die verarbeitet werden. Sie können mp4-Dateien ins Arbeitsverzeichnis laden. Außerdem können Sie YouTube-Videos per Link einfügen, die dann verarbeitet werden.

In [2]:
# Data-Ordner finden
with open("./.env", "r") as f:
    env = [line.replace("\n", "").split("=") for line in f.readlines()]
env = {e[0]: e[1] for e in env}
data_folder = env["INPUT_FOLDER"]
input_folder = os.path.join(data_folder, "input")
input_folder_parsed = Path(input_folder)
# YouTube-Videos suchen
if "input.txt" in os.listdir(input_folder):
    with open(os.path.join(input_folder, "input.txt"), "r") as f:
        yt_links = f.read()
else:
    yt_links = ""

# ---------- Widgets ----------

# Datei-Upload (nur mp4 zulassen)
upload_widget = widgets.FileUpload(
  accept='.mp4',
  multiple=True,
  description='MP4 hochladen'
)

# Mehrzeiliges Eingabefeld für URLs
url_textarea = widgets.Textarea(
  placeholder='Gib hier YouTube-Links ein (einer pro Zeile)',
  description='URLs:',
  layout=widgets.Layout(width='100%', height='100px'),
  value=yt_links
)

# Button zum Löschen aller MP4-Dateien
delete_button = widgets.Button(
  description='Alle MP4-Dateien löschen',
  button_style='danger'
)

# Bestätigungsbutton
confirm_button = widgets.Button(
  description='Auswahl bestätigen',
  button_style='success'
)

# Ausgabe für die Dateiliste
file_list_output = widgets.Output()

# ---------- Funktionen ----------

def save_uploaded_files(change):
    for fileinfo in upload_widget.value:
        file_path = input_folder_parsed / fileinfo["name"]
        with open(file_path, 'wb') as f:
            f.write(fileinfo['content'])
    upload_widget.value = []
    update_file_list()

def update_file_list():
  with file_list_output:
    clear_output()
    mp4_files = list(input_folder_parsed.glob('*.mp4'))
    if mp4_files:
      display(HTML('<b>Vorhandene MP4-Dateien:</b>'))
      for file in mp4_files:
        display(HTML(f'📁 {file.name}'))
    else:
      display(HTML('<i>Keine MP4-Dateien im Ordner.</i>'))

def delete_all_mp4_files(b):
  for file in input_folder_parsed.glob('*.mp4'):
    file.unlink()
  update_file_list()

def confirm_selection(b):
  urls = url_textarea.value.strip().splitlines()
  print('✅ Bestätigt:')
  print(f'- {len(list(input_folder_parsed.glob("*.mp4")))} MP4-Dateien im Ordner')
  print(f'- {len(urls)} URLs eingegeben')
  for url in urls:
    print(f'  • {url.strip()}')
  with open(input_folder_parsed / 'input.txt', 'w') as f:
      f.write("\n".join(urls))

# ---------- Event-Handler verbinden ----------

upload_widget.observe(save_uploaded_files, names='value')
delete_button.on_click(delete_all_mp4_files)
confirm_button.on_click(confirm_selection)

# ---------- UI anzeigen ----------

display(widgets.VBox([
  upload_widget,
  url_textarea,
  delete_button,
  confirm_button,
  file_list_output
]))

# Initiale Anzeige
update_file_list()

## 2. Ausführen der Skelettschätzung
Die Skelettschätzungen laufen auf Docker-Containern. Mit einem Klick auf den Button starten Sie die Ausführung.

⚠ Das kann eine Weile dauern.

In [3]:
def get_num_gpus():
    try:
        result = subprocess.run(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"],
                                capture_output=True,
                                text=True,
                                check=True)
        return len(result.stdout.strip().split("\n"))
    except:
        print("WARNUNG: Anzahl der GPUs konnte nicht ermittelt werden.")
        return 1

ray.init(num_gpus=get_num_gpus())

def run_container(service):
    print(f"⏳ Docker-Container läuft: {service}")
    container_name = f"{service}-{uuid.uuid4().hex[:8]}"
    result = subprocess.run(["docker", "compose", "run", "--no-deps", "--name", container_name, service])
    if result.returncode == 0:
        print(f"✔ Docker-Container beendet: {service}")
    else:
        print(f"❌ Fehler beim Ausführen von: {service} (Exit-Code: {result.returncode})")
        print(f"📜 Container-Logs von {container_name}:")
        subprocess.run(["docker", "logs", container_name])

@ray.remote(num_gpus=1)
def run_container_as_service(poser):
    run_container(poser)

def run(posers=None):
    if posers is None:
        posers = get_services()
    subprocess.run(["docker", "compose", "build"])
    run_container("input-loader")
    tasks = [run_container_as_service.remote(poser) for poser in posers]
    ray.get(tasks)
    run_container("codecs")
    run_container("data-transformer")

def get_services(docker_compose_path="./docker-compose.yml"):
    with open(docker_compose_path) as f:
        dc = yaml.load(f, Loader=yaml.FullLoader)
    services = list(dc["services"].keys())
    services.remove("input-loader")
    services.remove("data-transformer")
    services.remove("codecs")
    return services

# Widgets
status_output = widgets.Output()

def confirm(b):
    with status_output:
        clear_output()
        display(HTML('<p style="color:orange;font-weight:bold;">⏳ Programm wird ausgeführt ...</p>'))
    run()
    with status_output:
        clear_output(wait=True)
        display(HTML('<p style="color:darkgreen;font-weight:bold;">✅ Ausführung beendet</p>'))

# Button
confirm_button = widgets.Button(
  description='Programm starten',
  button_style='success'
)
confirm_button.on_click(confirm)

# Anzeige
display(widgets.VBox([confirm_button, status_output]))

2025-06-19 15:46:10,225	INFO worker.py:1752 -- Started a local Ray instance.


(run_container_as_service pid=1729142) ⏳ Docker-Container läuft: lightweight
(run_container_as_service pid=1729141) ⏳ Docker-Container läuft: blazepose


(run_container_as_service pid=1729142) time="2025-06-19T15:54:09+02:00" level=warning msg="/home/fese4852/Skelettschaetzer/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
(run_container_as_service pid=1729142) time="2025-06-19T15:54:09+02:00" level=warning msg="Found orphan containers ([input-loader-1bdcdbf4 data-transformer-5b20a341 codecs-b04841d6 openpifpaf-141890cc simcc-59df5151 debias-45d02c3b vipnas-686f8549 litehrnet-74858b87 rsn-fa62838b scnet-f6c772db mspn-3f9700d1 hrnetv2-9b2aee1e ipr-fbb25a2a hrnet-d275d116 dsnt-123ee08c simplebaseline2d-d841ec63 hourglass-1ff3a7e2 cpm-82c91081 deeppose-27fd1046 higher-hrnet-7d251c65 tfpose-98f2a556 lightweight-a3cb1558 blazepose-2ed7acce input-loader-e3d30f93 data-transformer-3e8af676 codecs-ad1f48a1 openpifpaf-54cc9a7f simcc-e72e800d debias-3d81d74d vipnas-304e927d litehrnet-0b919430 rsn-d615e6fd scnet-87f7d95e mspn-e5ec668a hrnetv2-d4f2a4bb ipr-29f5fb3c hrnet-661

(run_container_as_service pid=1729142) ✔ Docker-Container beendet: lightweight
(run_container_as_service pid=1731064) ⏳ Docker-Container läuft: tfpose


(run_container_as_service pid=1731064) time="2025-06-19T15:54:47+02:00" level=warning msg="/home/fese4852/Skelettschaetzer/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion" [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(run_container_as_service pid=1729141) time="2025-06-19T15:54:09+02:00" level=warning msg="Found orphan containers ([input-loader-1bdcdbf4 data-transformer-5b20a341 codecs-b04841d6 openpifpaf-141890cc simcc-59df5151 debias-45d02c3b vipnas-686f8549 litehrnet-74858b87 rsn-fa62838b scnet-f6c772db mspn-3f9700d1 hrnetv2-9b2aee1e ipr-fbb25a2a hrnet-d275d116 dsnt-123ee08c simplebaseline2d-d841ec63 hourglass-1ff3a7e2 cpm-82c91081 deeppose-27fd1046 higher-hrnet-7d251c65 tfpose-98f2a556 lightweight-a3cb1558 blazepose-2ed7acce input-l

...

(run_container_as_service pid=1762735) INFO:__main__:neural network device: cuda (CUDA available: True, count: 2)
(run_container_as_service pid=1762735) Verarbeite: /data/input/input_1.mp4
(run_container_as_service pid=1762735) INFO:__main__:neural network device: cuda (CUDA available: True, count: 2)


(run_container_as_service pid=1762735) Downloading: "http://github.com/vita-epfl/openpifpaf-torchhub/releases/download/v0.12.6/shufflenetv2k16-210404-110105-cocokp-o10s-f90ed364.pkl" to /root/.cache/torch/hub/checkpoints/shufflenetv2k16-210404-110105-cocokp-o10s-f90ed364.pkl
  0%|          | 0.00/39.3M [00:00<?, ?B/s] 
  3%|▎         | 1.10M/39.3M [00:00<00:03, 11.5MB/s]
  6%|▌         | 2.22M/39.3M [00:00<00:03, 11.6MB/s]
  8%|▊         | 3.34M/39.3M [00:00<00:03, 11.6MB/s]
 11%|█▏        | 4.47M/39.3M [00:00<00:03, 11.7MB/s]
 14%|█▍        | 5.59M/39.3M [00:00<00:03, 11.7MB/s]
 17%|█▋        | 6.72M/39.3M [00:00<00:02, 11.7MB/s]
 20%|█▉        | 7.84M/39.3M [00:00<00:02, 11.7MB/s]
 23%|██▎       | 8.97M/39.3M [00:00<00:02, 11.7MB/s]
 26%|██▌       | 10.1M/39.3M [00:00<00:02, 11.7MB/s]
 29%|██▊       | 11.2M/39.3M [00:01<00:02, 11.7MB/s]
 31%|███▏      | 12.3M/39.3M [00:01<00:02, 11.7MB/s]
 34%|███▍      | 13.5M/39.3M [00:01<00:02, 11.7MB/s]
 37%|███▋      | 14.6M/39.3M [00:01<00:02, 

(run_container_as_service pid=1762735) INFO:openpifpaf.predictor:Using multiple GPUs: 2
(run_container_as_service pid=1762735) INFO:openpifpaf.predictor:neural network device: cuda (CUDA available: True, count: 2)
(run_container_as_service pid=1762735) INFO:openpifpaf.show.animation_frame:video output = /data/OpenPifPaf/input_1.mp4
(run_container_as_service pid=1762735) INFO:openpifpaf.decoder.cifcaf:0 annotations: []
(run_container_as_service pid=1762735) INFO:openpifpaf.predictor:batch 0: no-file-name
(run_container_as_service pid=1762735) INFO:__main__:frame 1, loop time = 546ms (pre = 13.4ms, post = 4.1ms), FPS = 1.8
(run_container_as_service pid=1762735) INFO:openpifpaf.decoder.cifcaf:0 annotations: []
(run_container_as_service pid=1762735) INFO:openpifpaf.predictor:batch 1: no-file-name
(run_container_as_service pid=1762735) INFO:__main__:frame 2, loop time = 322ms (pre = 7.8ms, post = 277.5ms), FPS = 3.1
(run_container_as_service pid=1762735) INFO:openpifpaf.decoder.cifcaf:0 ann

(run_container_as_service pid=1762136) Downloading: "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth" to /root/.cache/torch/hub/checkpoints/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth


(run_container_as_service pid=1762735) INFO:openpifpaf.decoder.cifcaf:0 annotations: []
(run_container_as_service pid=1762735) INFO:openpifpaf.predictor:batch 491: no-file-name
(run_container_as_service pid=1762735) INFO:__main__:frame 492, loop time = 85ms (pre = 4.0ms, post = 55.2ms), FPS = 11.7
(run_container_as_service pid=1762735) INFO:openpifpaf.decoder.cifcaf:0 annotations: []
(run_container_as_service pid=1762735) INFO:openpifpaf.predictor:batch 492: no-file-name
(run_container_as_service pid=1762735) INFO:__main__:frame 493, loop time = 83ms (pre = 4.0ms, post = 53.5ms), FPS = 12.1
(run_container_as_service pid=1762735) INFO:openpifpaf.decoder.cifcaf:0 annotations: []
(run_container_as_service pid=1762735) INFO:openpifpaf.predictor:batch 493: no-file-name
(run_container_as_service pid=1762735) INFO:__main__:frame 494, loop time = 83ms (pre = 4.0ms, post = 53.0ms), FPS = 12.1
(run_container_as_service pid=1762735) INFO:openpifpaf.decoder.cifcaf:0 annotations: []
(run_container_

Statusmeldungen während der Ausführung der Container können Sie sich in den Logs ansehen:
`View > Show Log Console`